In [34]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split, KFold
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PowerTransformer 
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.metrics import mean_squared_error
from sklearn import set_config
from lightgbm import LGBMRegressor
import lightgbm as lgb
import optuna
from optuna.distributions import CategoricalDistribution, IntDistribution, FloatDistribution
from optuna.integration import OptunaSearchCV, ShapleyImportanceEvaluator
from optuna.integration.lightgbm import LightGBMTunerCV #lgbm 전용 tunner 가 존재
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeClassifier
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn import model_selection
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler

## 데이터 불러오기

In [35]:
X_train1 = pd.read_csv('X_train.csv', encoding='cp949')
y_train1 = pd.read_csv('y_train.csv', encoding='cp949')

In [36]:
train=pd.merge(X_train1, y_train1, on='ID')

In [37]:
X_train = pd.read_csv('X_train.csv', encoding='cp949').drop(columns='ID')
y_train = pd.read_csv('y_train.csv', encoding='cp949').Salary

X_test = pd.read_csv('X_test.csv', encoding='cp949')
test_id = X_test.ID
X_test = X_test.drop(columns='ID')

## 결측지 확인

In [38]:
#'직무태그','근무지역','근무형태'

# 결측지 제거
## 1) 직무태그  

결측치 처리: 없음

In [39]:
X_train['직무태그']=X_train['직무태그'].fillna('없음')
X_test['직무태그']=X_test['직무태그'].fillna('없음')

## 2) 근무형태  

결측치: 계약직으로 대체  


In [40]:
# 결측치 계약직으로 대체
X_train['근무형태']=X_train['근무형태'].fillna('계약직')
X_test['근무형태']=X_test['근무형태'].fillna('계약직')

## 3) 어학시험

어학시험  
결측치: 불확실로 처리  
피처 수정: 기타시험, 기타 -> 기타로 묶기  


In [41]:
X_train['어학시험']=X_train['어학시험'].fillna('미응시')
X_test['어학시험']=X_test['어학시험'].fillna('미응시')

## 3) 대학성적  
결측치: 중앙값 대체

In [42]:
check = train.copy()
check['대학성적'] = check['대학성적'].fillna('결측지')

In [43]:
check.groupby('대학성적')['Salary'].mean()

대학성적
0.0      2887.500000
30.0     2600.000000
40.0     2705.434783
50.0     2658.419958
60.0     2798.009650
70.0     2835.310465
80.0     2794.408840
90.0     2759.366197
100.0    2371.428571
110.0    3834.375000
결측지      2858.781726
Name: Salary, dtype: float64

In [44]:
# 결측지의 target평균이 중앙값인 70과 유사하므로 중앙값으로 대체
X_train['대학성적']= X_train['대학성적'].fillna(X_train['대학성적'].median())
X_test['대학성적']= X_test['대학성적'].fillna(X_train['대학성적'].median())

## 최종확인

In [45]:
X_train.isnull().sum().sum()

0

In [46]:
X_test.isnull().sum().sum()

0

# 피처 전처리

### 1) 직종

In [47]:
# 처리안함

### 2) 세부직종

In [48]:
X_train['세부직종'].nunique()

109

### 3) 직무태그

In [49]:
# , 기준으로 나누기
X_train['직무태그']= X_train['직무태그'].str.replace('·',',')
X_test['직무태그']= X_test['직무태그'].str.replace('·',',')

# X_train['직무태그']= X_train['직무태그'].str.replace('/',',')
# X_test['직무태그']= X_test['직무태그'].str.replace('/',',')


# X_train['직무태그']= X_train['직무태그'].str.replace('-',',')
# X_test['직무태그']= X_test['직무태그'].str.replace('-',',')

# X_train['직무태그']= X_train['직무태그'].str.replace('&',',')
# X_test['직무태그']= X_test['직무태그'].str.replace('&',',')

# X_train['직무태그']= X_train['직무태그'].str.replace('_',',')
# X_test['직무태그']= X_test['직무태그'].str.replace('_',',')

In [50]:
# 빈칸없애기
X_train['직무태그']= X_train['직무태그'].str.replace(' ','')
X_test['직무태그']= X_test['직무태그'].str.replace(' ','')

# 맨뒷글자 ,인경우 없애기
a = []
for i in X_train['직무태그']:
    if i[-1] == ',':
        a.append(i[:-1])
    elif i[-1] == '.':
        a.append(i[:-1])
    elif i[0] == ',':
        a.append(i[1:])
    else:
        a.append(i)
    
X_train['직무태그'] = a

b = []
for j in X_test['직무태그']:
    if j[-1] == ',':
        b.append(j[:-1])
    elif j[-1] == '.':
        b.append(j[:-1])
    elif j[0] == ',':
        b.append(j[1:])
    else:
        b.append(j)
    
X_test['직무태그'] = b

gik_lst = ['#53245','#49406']


for i in gik_lst:
    X_train.loc[X_train['직무태그'].str.contains(f'{i}'),'직무태그'] = '기타'
    X_test.loc[X_test['직무태그'].str.contains(f'{i}'), '직무태그'] = '기타'
    
    
    
X_train['직무태그'].loc[X_train['직무태그'] == '기획MD'] = '기획md'
X_train['직무태그'].loc[X_train['직무태그'] == '디지이너'] = '디자이너'
X_train['직무태그'].loc[X_train['직무태그'] == '공무부'] = '공무원'
X_train['직무태그'].loc[X_train['직무태그'] == '관리자'] = '관리직'
X_train['직무태그'].loc[X_train['직무태그'] == '기술자'] = '기술직'
X_train['직무태그'].loc[X_train['직무태그'] == '웹개발'] = '웹기획'
X_train['직무태그'].loc[X_train['직무태그'] == '전산실'] = '전산직'
X_train['직무태그'].loc[X_train['직무태그'] == '회계외'] = '회계사'

X_train['직무태그'].loc[X_train['직무태그'] == '칼라리스트'] = '컬러리스트'
X_train['직무태그'].loc[X_train['직무태그'] == '텔레마케터'] = '텔레마케팅'
X_train['직무태그'].loc[X_train['직무태그'] == '프랜차이즈'] = '프렌차이즈'
X_train['직무태그'].loc[X_train['직무태그'] == 'Salesadmin'] = 'SalesAdmin'
X_train['직무태그'].loc[X_train['직무태그'] == '매니저'] = '메니져'





X_test['직무태그'].loc[X_test['직무태그'] == '기획MD'] = '기획md'
X_test['직무태그'].loc[X_test['직무태그'] == '디지이너'] = '디자이너'
X_test['직무태그'].loc[X_test['직무태그'] == '공무부'] = '공무원'
X_test['직무태그'].loc[X_test['직무태그'] == '관리자'] = '관리직'
X_test['직무태그'].loc[X_test['직무태그'] == '기술자'] = '기술직'
X_test['직무태그'].loc[X_test['직무태그'] == '웹개발'] = '웹기획'
X_test['직무태그'].loc[X_test['직무태그'] == '전산실'] = '전산직'
X_test['직무태그'].loc[X_test['직무태그'] == '회계외'] = '회계사'

X_test['직무태그'].loc[X_test['직무태그'] == '칼라리스트'] = '컬러리스트'
X_test['직무태그'].loc[X_test['직무태그'] == '텔레마케터'] = '텔레마케팅'
X_test['직무태그'].loc[X_test['직무태그'] == '프랜차이즈'] = '프렌차이즈'
X_test['직무태그'].loc[X_test['직무태그'] == 'Salesadmin'] = 'SalesAdmin'
X_test['직무태그'].loc[X_test['직무태그'] == '매니저'] = '메니져'

gik_no = ['시원하게따사롭게','7277','영업,영없관리','없음','소리없이끈기있는...','생산관리분야에관한경력은없지만기회가있으면근무하고싶습니다',
        '8483','2172','aaaa','패기.열정이있는젊은이!나를선택해주십시요!','ㅇㅇ','12ㅂㅈ', 'ㅁㅁ', 'ㄷ','회사의발전이곧나의발전이라생각하며열심히일하겠습니다.',
        '여러모임에서의회장을역임한경험이큰발전의계기가될것입니다.','LGDisplay신공장기획및국내영업경력이력서입니다','자금조달,운용,집행및총무,인사업무경력자입니다.',
         '능력있는전자구매자입니다.','열심히노력하는RF엔지니어입니다','서인호이력서입니다','준비된자세한상민입니다','이제3년차에접어든재무요원입니다...','난감동적인사람이다',
         '포기는김장철에나쓰는말이다.','불가능그것은도전이라는이름앞에무너지는것이다.','TOEIC940,무역전공,신입,해외영업을하고싶습니다.','무','.','p','웹','일','1','법','C','??',
        '금웅','금용','급여','성찰','성실','조선','중국','지원','희망','현지','429','멋쟁이','새출발','신동영','심응재','써비스','윤재임','jw134','alsl1'
        '조주현','준비중','최영연','최희주','1052','1201','1878','7770','test','pc가능','박미선','보상과','중국통','변화무쌍','트레이너','11111','준비된비서',
         '#NAME?','kj0121','cccccc','gun3733','70013867','bong9411','농산물Buyer','영업밖에모르는놈','21세기새로운도전','능력있는영업지원자','무엇이든할수있어요',
        '소리없이끈기있는..','회계담당자...경리','회계의모든것은나에게로','사무,총부,인사....','회계,세무분야성실한인재',',영어,일본어,번역,통역','1금융권에서10년이상근무',
         '성실하고능력있는웹디자이너','★언제나처음과같은마음으로★','김세미의해외영업직지원이력서','해외영업분야인재를찾고계십니까',"반도체,공정eng'r,Engineer,진공",'최선을다한나',
         '제가필요로하는곳.','일러스트,,#53245;,포토,#49406;,드림위버,기획력',',','0','?','s','ㅇ','영','!!','6σ','11','임대호',',,,','207','310','간다잉','금융外','이진혁',
         '3240','4831','9336','ho30','비전21','나의이력서','bar,까페','내가해결하라','최영철이력서','yesu153','뛰는영업관리자','leejs718','후회없는삶을살자','강력한서비스마인드','#49406;매니저',
         ]
          

for i in X_train.loc[X_train['직무태그'].str.contains('니다'), '직무태그']:
    gik_no.append(i)
    
for i in X_train.loc[X_train['직무태그'].str.contains('!'), '직무태그']:
    gik_no.append(i)
    

X_train['직무태그'] = X_train['직무태그'].replace(gik_no, '기타')
X_test['직무태그'] = X_test['직무태그'].replace(gik_no, '기타')

X_train['직무태그'].nunique()

### 4) 근무경력

In [51]:
a = []
X_train['근무경력_원본'] = X_train['근무경력'].copy()
for i in X_train['근무경력']:
    split = i.split(' ')
    if len(split) == 2:
        a.append(int(split[0][:-1])*12 + int(split[1][:-2]))
    else: a.append(int(split[0][:-2]))

X_train['근무경력'] = a

b = []
X_test['근무경력_원본'] = X_test['근무경력'].copy()
for j in X_test['근무경력']:
    split_2 = j.split(' ')
    if len(split_2) == 2:
        b.append(int(split_2[0][:-1])*12 + int(split_2[1][:-2]))
    else: b.append(int(split_2[0][:-2]))

X_test['근무경력'] = b

### 5) 근무형태

In [52]:
# 근무형태 피처를 게약직, 정규직으로 이분화 하기전에 해외취업여부를 미리 뽑아내야되기 떄문에 피처만들기 단계에서 할 일을 미리해둠
a = []
for i in X_train['근무형태']:
    if '해외취업' in i:
        a.append('o')
    else: a.append('x')

X_train['해외취업여부'] = a

b = []
for j in X_test['근무형태']:
    if '해외취업' in j:
        b.append('o')
    else: b.append('x')

X_test['해외취업여부'] = b

In [53]:
# # 인턴이 포함된 데이터들의 타겟의 평균이 눈에띄게 낮기떄문에 인턴들을 먼저 계약직으로 분류
# a = []
# for i in X_train['근무형태']:
#     if '인턴' in i:
#         a.append('계약직')
#     elif '정규직' in i:
#         a.append('정규직')
#     else: a.append('계약직')

# X_train['근무형태'] = a

# b = []
# for j in X_test['근무형태']:
#     if '인턴' in j:
#         b.append('계약직')
#     elif '정규직' in j:
#         b.append('정규직')
#     else: b.append('계약직')

# X_test['근무형태'] = b

### 6) 근무지역

In [54]:
X = pd.concat([X_train,X_test], ignore_index=True)

In [55]:
region_lst=[]
for val in X['근무지역']:
    region_lst.append(val.split(','))

In [56]:
X['근무지역1'] = np.zeros(shape=(X.shape[0],), dtype=np.int8)
X['근무지역2'] = np.zeros(shape=(X.shape[0],), dtype=np.int8)
X['근무지역3'] = np.zeros(shape=(X.shape[0],), dtype=np.int8)

In [57]:
for k in range(0, len(region_lst)):
    cnt = 0
    X['근무지역1'][k] = region_lst[k][cnt]
    cnt += 1
    if region_lst[k][cnt] == '' and region_lst[k][cnt+1] == '':
        X['근무지역2'][k] = X['근무지역1'][k]
        X['근무지역3'][k] = X['근무지역1'][k]
    elif region_lst[k][cnt+1] == '' and region_lst[k][cnt] != '':
        X['근무지역2'][k] = X['근무지역1'][k]
        X['근무지역3'][k] = X['근무지역2'][k]
    else:
        X['근무지역2'][k] = region_lst[k][cnt]
        X['근무지역3'][k] = region_lst[k][cnt+1]

In [58]:
#X.drop('근무지역', axis=1, inplace=True)

In [59]:
X_train = X.iloc[:X_train.shape[0],:].reset_index(drop=True)
X_test = X.iloc[X_train.shape[0]:,:].reset_index(drop=True)

### 7) 출신대학

### 8) 대학전공

In [60]:
#### 맨뒷글자가 '과' 인경우 '~~'학과 또는 '~~'과 이기때문에 '~~'와 같이 분류할 수 있다고 판단함
a = []
for i in X_train['대학전공']:
    if i[-1] == '과':
        a.append(i[:-1])
    else:
        a.append(i)
    
X_train['대학전공'] = a

b = []
for j in X_test['대학전공']:
    if j[-1] == '과':
        b.append(j[:-1])
    else:
        b.append(j)
    
X_test['대학전공'] = b

In [61]:
#### 대학전공에 빈칸(스페이스)를 없앴습니다
a = []
for i in X_train['대학전공']:
    a.append(i.replace(' ', ''))
    
X_train['대학전공'] = a    
    

b=[]    
for i in X_test['대학전공']:
    b.append(i.replace(' ', ''))
    
X_test['대학전공'] = b

In [62]:
uni_lst = ['중국','일본','영어','국어','경영','경제', '행정','회계', '정보','컴퓨터','기계','전기','건축','수학', '통계','생명','생물',
'전자','미술', '체육', '음악','교육','디자인','화학', '정치', '환경', '법','소프트웨어','철학', '의류', '불어','물리','동양화','무역','금속',
'전산','사학','산업','토목','독어','식품','광고','재료','토목','제어계측','유전','도시공학','국제통상','심리','사회학','신소재','공예','사회학',
'관광개발','멀티미디어','디지털컨텐츠','전파공학','국제관광','연극영화','문예창작','응용소재','사범대','문화컨테츠','지역사회과학','항공우주',
 '관광', '광통신','사회복지','도시','천문','E-','신문','자원','전파','가정','제어','고분자','아동','조형','동양어','지역','레저',
'생활과학','스포츠','섬유','영상','연극','연예','건설','자연','조리','인터넷','해양','호텔','영미','지구','문예','자동차','태권도',
'원예','유통','경호','운동','러시아','디지털','신학','메체','스페인','회화','네트워크','생산','무용','사회','민속','국제','가족',
'프랑스','예술','텍스타일','역사','사진','문화재','서양','미디어','부동산','기악','보험','금융','마케팅','에너지','청소년','영양',
 '선교', '통신','소비자','건강','제약','낙농학','매체','유학','윤리','보육','격기','지도','병리','세라믹','조소','농','인류','재료',
  '메카','유도','보건','조경','시각','성악','경찰','웹프로그램','게임','사범','복지','한국','도서','재무','상업','언론','시스템',
'애니']


for i in uni_lst:
    X_train.loc[X_train['대학전공'].str.contains(f'{i}'),'대학전공'] = f'{i}'
    X_test.loc[X_test['대학전공'].str.contains(f'{i}'), '대학전공'] = f'{i}'
    
#반복문으로 한번에 안되는 것들 따로 설정 

X_train.loc[X_train['대학전공'].str.contains('중문'), '대학전공'] = '중국'
X_train.loc[X_train['대학전공'].str.contains('Chinese'), '대학전공'] = '중국'
X_train.loc[X_train['대학전공'].str.contains('중어'), '대학전공'] = '중국'
X_train.loc[X_train['대학전공'].str.contains('일어'), '대학전공'] = '일본'
X_train.loc[X_train['대학전공'].str.contains('일문학'), '대학전공'] = '일본'
X_train.loc[X_train['대학전공'].str.contains('영문'), '대학전공'] = '영어'
X_train.loc[X_train['대학전공'].str.contains('국문'), '대학전공'] = '국어'
X_train.loc[X_train['대학전공'].str.contains('의상'),'대학전공'] = '의류'
X_train.loc[X_train['대학전공'].str.contains('Management'),'대학전공'] = '경영'
X_train.loc[X_train['대학전공'].str.contains('BUSINESSADMINISTRATION'),'대학전공'] = '경영'
X_train.loc[X_train['대학전공'].str.contains('스포츠레져'),'대학전공'] = '스포츠레저'
X_train.loc[X_train['대학전공'].str.contains('컴공'), '대학전공'] = '컴퓨터'
X_train.loc[X_train['대학전공'].str.contains('e-'), '대학전공'] = 'E-'
X_train.loc[X_train['대학전공'].str.contains('E비즈'), '대학전공'] = 'E-'
X_train.loc[X_train['대학전공'].str.contains('e비즈'), '대학전공'] = 'E-'
X_train.loc[X_train['대학전공'].str.contains('레져'), '대학전공'] = '레저'
X_train.loc[X_train['대학전공'].str.contains('Chemistry'), '대학전공'] = '화학'
X_train.loc[X_train['대학전공'].str.contains('Music'), '대학전공'] = '음악'
X_train.loc[X_train['대학전공'].str.contains('HOTEL'), '대학전공'] = '호텔'
X_train.loc[X_train['대학전공'].str.contains('패션'), '대학전공'] = '디자인'
X_train.loc[X_train['대학전공'].str.contains('Design'), '대학전공'] = '디자인'
X_train.loc[X_train['대학전공'].str.contains('물류'), '대학전공'] = '유통'
X_train.loc[X_train['대학전공'].str.contains('의생활'), '대학전공'] = '의류'
X_train.loc[X_train['대학전공'].str.contains('사하'), '대학전공'] = '역사'
X_train.loc[X_train['대학전공'].str.contains('E비지'), '대학전공'] = 'E-'
X_train.loc[X_train['대학전공'].str.contains('e비지'), '대학전공'] = 'E-'
X_train.loc[X_train['대학전공'].str.contains('Englishlanguage&literature'), '대학전공'] = '영어'
X_train.loc[X_train['대학전공'].str.contains('이비즈니스'), '대학전공'] = 'E-'
X_train.loc[X_train['대학전공'].str.contains('이비즈니스학'), '대학전공'] = 'E-'
X_train.loc[X_train['대학전공'].str.contains('이비지니스학'), '대학전공'] = 'E-'
X_train.loc[X_train['대학전공'].str.contains('PublicAdministration'), '대학전공'] = '행정'
X_train.loc[X_train['대학전공'].str.contains('노문학'), '대학전공'] = '러시아'
X_train.loc[X_train['대학전공'].str.contains('노어'), '대학전공'] = '러시아'
X_train.loc[X_train['대학전공'].str.contains('불문'), '대학전공'] = '프랑스'
X_train.loc[X_train['대학전공'].str.contains('독문'), '대학전공'] = '독어'
X_train.loc[X_train['대학전공'].str.contains('F-Horn'), '대학전공'] = '음악'
X_train.loc[X_train['대학전공'].str.contains('소트웨어'), '대학전공'] = '소프트웨어'
X_train.loc[X_train['대학전공'].str.contains('MaterialsEngineering'), '대학전공'] = '재료공학'
X_train.loc[X_train['대학전공'].str.contains('InternationalTrade'), '대학전공'] = '무역'
X_train.loc[X_train['대학전공'].str.contains('clothingandtextile'), '대학전공'] = '의류'



X_test.loc[X_test['대학전공'].str.contains('중문'), '대학전공'] = '중국'
X_test.loc[X_test['대학전공'].str.contains('Chinese'), '대학전공'] = '중국'
X_test.loc[X_test['대학전공'].str.contains('중어'), '대학전공'] = '중국'
X_test.loc[X_test['대학전공'].str.contains('일어'), '대학전공'] = '일본'
X_test.loc[X_test['대학전공'].str.contains('일문학'), '대학전공'] = '일본'
X_test.loc[X_test['대학전공'].str.contains('영문'), '대학전공'] = '영어'
X_test.loc[X_test['대학전공'].str.contains('국문'), '대학전공'] = '국어'
X_test.loc[X_test['대학전공'].str.contains('의상'),'대학전공'] = '의류'
X_test.loc[X_test['대학전공'].str.contains('Management'),'대학전공'] = '경영'
X_test.loc[X_test['대학전공'].str.contains('BUSINESSADMINISTRATION'),'대학전공'] = '경영'
X_test.loc[X_test['대학전공'].str.contains('e-'), '대학전공'] = 'E-'
X_test.loc[X_test['대학전공'].str.contains('E비즈니스'), '대학전공'] = 'E-'
X_test.loc[X_test['대학전공'].str.contains('e비즈'), '대학전공'] = 'E-'
X_test.loc[X_test['대학전공'].str.contains('레져'), '대학전공'] = '레저'
X_test.loc[X_test['대학전공'].str.contains('Chemistry'), '대학전공'] = '화학'
X_test.loc[X_test['대학전공'].str.contains('Music'), '대학전공'] = '음악'
X_test.loc[X_test['대학전공'].str.contains('HOTEL'), '대학전공'] = '호텔'
X_test.loc[X_test['대학전공'].str.contains('패션'), '대학전공'] = '디자인'
X_test.loc[X_test['대학전공'].str.contains('Design'), '대학전공'] = '디자인'
X_test.loc[X_test['대학전공'].str.contains('물류'), '대학전공'] = '유통'
X_test.loc[X_test['대학전공'].str.contains('의생활'), '대학전공'] = '의류'
X_test.loc[X_test['대학전공'].str.contains('사하'), '대학전공'] = '역사'
X_test.loc[X_test['대학전공'].str.contains('E비지'), '대학전공'] = 'E-'
X_test.loc[X_test['대학전공'].str.contains('e비지'), '대학전공'] = 'E-'
X_test.loc[X_test['대학전공'].str.contains('Englishlanguage&literature'), '대학전공'] = '영어'
X_test.loc[X_test['대학전공'].str.contains('이비즈니스'), '대학전공'] = 'E-'
X_test.loc[X_test['대학전공'].str.contains('이비즈니스학'), '대학전공'] = 'E-'
X_test.loc[X_test['대학전공'].str.contains('이비지니스학'), '대학전공'] = 'E-'
X_test.loc[X_test['대학전공'].str.contains('PublicAdministration'), '대학전공'] = '행정'
X_test.loc[X_test['대학전공'].str.contains('노문학'), '대학전공'] = '러시아'
X_test.loc[X_test['대학전공'].str.contains('노어'), '대학전공'] = '러시아'
X_test.loc[X_test['대학전공'].str.contains('불문'), '대학전공'] = '프랑스'
X_test.loc[X_test['대학전공'].str.contains('독문'), '대학전공'] = '독어'
X_test.loc[X_test['대학전공'].str.contains('F-Horn'), '대학전공'] = '음악'
X_test.loc[X_test['대학전공'].str.contains('소트웨어'), '대학전공'] = '소프트웨어'
X_test.loc[X_test['대학전공'].str.contains('MaterialsEngineering'), '대학전공'] = '재료공학'
X_test.loc[X_test['대학전공'].str.contains('InternationalTrade'), '대학전공'] = '무역'
X_test.loc[X_test['대학전공'].str.contains('clothingandtextile'), '대학전공'] = '의류'

In [63]:
### unique에 오타로 판단되는 부분과 전공으로 판단되지 않는 부분을 '오타'로 대체했습니다.
ss = ['1', '졸업','전공','토&#47791공학', 'e-businees학', '산', '영','멀티미디어학부(인터넷응요)','InternationalSummerProgram',
      '어학연수','ㅇ','ㅇㅇ', '조리외식졍영','사','철철','토옥','철','새왈과학부','외식사업','영','문화컨테츠',
      '사체','eh', '전공', 'dddddd', '1', '3.03', 'ghj','주거솬경학','회힉공학','Theatre','경역학','체욱학','문화제보존학','건충공학'
     ,'기게공학','&#49887;품공학','law','수원대학교3학년편입후졸업','ㅁㅇㄻㅈㄷㄹ','physics','없음','MIS','관공통역','수원대학교',
     'SocialHumanMovementandperformance','관좡학','library&info.science','환격공학','CommunicationConsulting','핼정학','마이컴',
     '웨이트트레이닝','이공계열중퇴','화확공학']

X_train = X_train.replace(ss, '기타')
X_test = X_test.replace(ss, '기타')


### 9) 어학시험

In [64]:
# ' '값을 가지는 데이터들을 결측지 즉 미응시로 대체하려 했으나 그 값들의 타겟의 평균의 매우 높아 불확실로 따로 분류 
X_train['어학시험'].replace(' ', '불확실', inplace=True)
X_test['어학시험'].replace(' ', '불확실', inplace=True)

In [65]:
# 기타시험과 기타를 같이 분류
X_train['어학시험'].replace('기타시험','기타',inplace=True)
X_train['어학시험'].replace('기타시험','기타',inplace=True)

In [66]:
X_train.shape

(16570, 16)

In [67]:
X_test.shape

(11048, 16)

### 10) 자격증

### 11) 대학성적

## csv파일로 내보내기

In [68]:
X_train.to_csv('X_train_candidate1.csv', index=False)

In [69]:
X_test.to_csv('X_test_candidate1.csv', index=False)